<a href="https://colab.research.google.com/github/kurehajime/openapi_sandbox/blob/main/%E6%A0%B9%E6%8B%A0%E4%BB%98%E3%81%8DQ%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

根拠付きQ&A
---
このノートブックはGPU環境でのみ実行できます。
次のブロックでOpen AIのAPIキーを入力してください。

In [13]:
from getpass import getpass
secret = getpass('Enter the secret value: ')  # APIキーの取得

Enter the secret value: ··········


In [14]:
!pip install openai 
!pip install faiss-gpu -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
qa1 = """Q. 2023年2月24日の鹿児島の天気は？
A. 雨のち晴れです。
"""
qa2 = """Q. kurehajimeはうまい棒でどの味が好きですか？
A. 納豆味が好きです。
"""
qa3 = """Q. 2023年02月9日～10日に行われた王将戦第4局ではどちらが勝ちましたか？
A. 羽生善治九段が藤井聡太王将に勝ちました。
"""

In [16]:
import openai
openai.api_key = secret
sentences = [qa1,qa2,qa3]
sentences = [x.replace("\n", " ") for x in sentences]
embeddings = openai.Embedding.create(input = sentences, model="text-embedding-ada-002")['data']

In [17]:
import faiss
import numpy as np
d = 1536
index = faiss.IndexFlatL2(d)
embeddings = np.array([x["embedding"] for x in embeddings], dtype=np.float32) 
index.add(embeddings) 

In [18]:
query = "2023年の王将戦第4局では誰が誰に負けましたか？"
query_embedding = np.array([openai.Embedding.create(input = query , model="text-embedding-ada-002")['data'][0]["embedding"]], dtype=np.float32)
k = 1
D, I = index.search(query_embedding, k)
hint = sentences[I[0][0]]
print(f"hint: {hint}")

hint: Q. 2023年02月9日～10日に行われた王将戦第4局ではどちらが勝ちましたか？ A. 羽生善治九段が藤井聡太王将に勝ちました。 


In [19]:
answer = openai.Completion.create(engine="text-davinci-003", prompt=f"hint:\n {hint}\n\n---\nQ: {query}\n", max_tokens=256)["choices"][0]["text"]
print(f"Q: {query}")
print(answer)

Q: 2023年の王将戦第4局では誰が誰に負けましたか？
A: 藤井聡太王将が羽生善治九段に負けました。
